## With Write Back and Write Through 

In [1]:
import tkinter as tk
from tkinter import filedialog

def read_memory_addresses(filename):
    addresses = []
    with open(filename, 'r') as file:
        for line in file:
            address = int(line, 16)
            addresses.append(address)
    return addresses

def simulate_cache(cache_size, block_size, associativity, memory_addresses, write_policy):
    num_sets = (cache_size * 1024) // (block_size * associativity)
    cache = [{'state': 'I', 'data': None} for _ in range(num_sets)]
    output_summary = []
    read_cache_hits = 0
    read_cache_misses = 0
    write_cache_hits = 0
    write_cache_misses = 0

    for address in memory_addresses:
        block_index = address // block_size
        set_index = block_index % num_sets
        cache_line = cache[set_index]

        if cache_line['state'] == 'I':
            output_summary.append(f"Read: Address {hex(address)} - Cache Miss")
            cache_line['data'] = "Data from Memory"
            cache_line['state'] = 'S'  # Transition to Shared state
            read_cache_misses += 1
        elif cache_line['state'] == 'S':
            output_summary.append(f"Read: Address {hex(address)} - Cache Hit (Shared)")
            read_cache_hits += 1
        elif cache_line['state'] == 'M':
            output_summary.append(f"Read: Address {hex(address)} - Cache Hit (Modified)")
            read_cache_hits += 1

        if write_policy == 'WB':
            write_address = address
            write_block_index = write_address // block_size
            write_set_index = write_block_index % num_sets
            write_cache_line = cache[write_set_index]

            if write_cache_line['state'] == 'I':
                output_summary.append(f"Write: Address {hex(write_address)} - Cache Miss")
                write_cache_line['data'] = "Data from Memory"
                write_cache_line['state'] = 'M'  # Transition to Modified state
                write_cache_misses += 1
            elif write_cache_line['state'] == 'S':
                output_summary.append(f"Write: Address {hex(write_address)} - Cache Miss")
                write_cache_line['data'] = "Data from Memory"
                write_cache_line['state'] = 'M'  # Transition to Modified state
                write_cache_misses += 1
            elif write_cache_line['state'] == 'M':
                output_summary.append(f"Write: Address {hex(write_address)} - Cache Hit (Modified)")
                write_cache_hits += 1

        elif write_policy == 'WT':
            output_summary.append(f"Write: Address {hex(address)} - Cache Miss (Write-Through)")
            write_cache_misses += 1

    read_cache_hit_rate = read_cache_hits / (read_cache_hits + read_cache_misses)
    read_cache_miss_rate = read_cache_misses / (read_cache_hits + read_cache_misses)
    write_cache_hit_rate = write_cache_hits / (write_cache_hits + write_cache_misses)
    write_cache_miss_rate = write_cache_misses / (write_cache_hits + write_cache_misses)

    return output_summary, read_cache_hit_rate, read_cache_miss_rate, write_cache_hit_rate, write_cache_miss_rate

def run_simulation():
    cache_size = int(cache_size_entry.get())
    block_size = int(block_size_entry.get())
    associativity = int(associativity_entry.get())
    write_policy = write_policy_var.get()
    file_path = file_path_var.get()
    
    try:
        memory_addresses = read_memory_addresses(file_path)
        output_summary, read_hit_rate, read_miss_rate, write_hit_rate, write_miss_rate = simulate_cache(cache_size, block_size, associativity, memory_addresses, write_policy)
        result_text.config(state="normal")
        result_text.delete("1.0", "end")
        result_text.insert("end", "Simulation Summary:\n")
        result_text.insert("end", f"Read Cache Hit Rate: {read_hit_rate:.2%}\n")
        result_text.insert("end", f"Read Cache Miss Rate: {read_miss_rate:.2%}\n")
        result_text.insert("end", f"Write Cache Hit Rate: {write_hit_rate:.2%}\n")
        result_text.insert("end", f"Write Cache Miss Rate: {write_miss_rate:.2%}\n\n")
        for line in output_summary:
            result_text.insert("end", line + "\n")
        result_text.config(state="disabled")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {str(e)}")

app = tk.Tk()
app.title("MSI Cache Simulator")
app.geometry("800x600")

cache_size_label = tk.Label(app, text="Cache Size (KB):")
cache_size_entry = tk.Entry(app)
block_size_label = tk.Label(app, text="Block Size (bytes):")
block_size_entry = tk.Entry(app)
associativity_label = tk.Label(app, text="Associativity:")
associativity_entry = tk.Entry(app)

file_label = tk.Label(app, text="Select Memory Addresses File:")
file_path_var = tk.StringVar()
file_entry = tk.Entry(app, textvariable=file_path_var, state="readonly")
file_browse_button = tk.Button(app, text="Browse", command=lambda: file_path_var.set(filedialog.askopenfilename()))

write_policy_label = tk.Label(app, text="Write Policy:")
write_policy_var = tk.StringVar(value="WB")
write_policy_wb = tk.Radiobutton(app, text="Write-Back (WB)", variable=write_policy_var, value="WB")
write_policy_wt = tk.Radiobutton(app, text="Write-Through (WT)", variable=write_policy_var, value="WT")

simulate_button = tk.Button(app, text="Simulate", command=run_simulation)

result_text = tk.Text(app, state="disabled", height=27, width=88)
result_text.grid(row=6, column=0, columnspan=3)

cache_size_label.grid(row=0, column=0)
cache_size_entry.grid(row=0, column=1)
block_size_label.grid(row=1, column=0)
block_size_entry.grid(row=1, column=1)
associativity_label.grid(row=2, column=0)
associativity_entry.grid(row=2, column=1)
file_label.grid(row=3, column=0)
file_entry.grid(row=3, column=1)
file_browse_button.grid(row=3, column=2)
write_policy_label.grid(row=4, column=0)
write_policy_wb.grid(row=4, column=1)
write_policy_wt.grid(row=4, column=2)
simulate_button.grid(row=5, column=0, columnspan=3)

app.mainloop()


## With Replacement Policy 

In [2]:
import tkinter as tk
from tkinter import filedialog
from collections import deque

def read_memory_addresses(filename):
    addresses = []
    with open(filename, 'r') as file:
        for line in file:
            address = int(line, 16)
            addresses.append(address)
    return addresses

def simulate_cache(cache_size, block_size, associativity, memory_addresses, write_policy, replacement_policy):
    num_sets = (cache_size * 1024) // (block_size * associativity)
    cache = [{'state': 'I', 'data': None} for _ in range(num_sets)]
    cache_order = deque(maxlen=num_sets)  # Used to maintain cache line order for replacement
    output_summary = []
    read_cache_hits = 0
    read_cache_misses = 0
    write_cache_hits = 0
    write_cache_misses = 0

    for address in memory_addresses:
        block_index = address // block_size
        set_index = block_index % num_sets
        cache_line = cache[set_index]

        if cache_line['state'] == 'I':
            output_summary.append(f"Read: Address {hex(address)} - Cache Miss")
            cache_line['data'] = "Data from Memory"
            cache_line['state'] = 'S'  # Transition to Shared state
            read_cache_misses += 1
        elif cache_line['state'] == 'S':
            output_summary.append(f"Read: Address {hex(address)} - Cache Hit (Shared)")
            read_cache_hits += 1
        elif cache_line['state'] == 'M':
            output_summary.append(f"Read: Address {hex(address)} - Cache Hit (Modified)")
            read_cache_hits += 1

        if write_policy == 'WB':
            write_address = address
            write_block_index = write_address // block_size
            write_set_index = write_block_index % num_sets
            write_cache_line = cache[write_set_index]

            if write_cache_line['state'] == 'I':
                output_summary.append(f"Write: Address {hex(write_address)} - Cache Miss")
                write_cache_line['data'] = "Data from Memory"
                write_cache_line['state'] = 'M'  # Transition to Modified state
                write_cache_misses += 1
            elif write_cache_line['state'] == 'S':
                output_summary.append(f"Write: Address {hex(write_address)} - Cache Miss")
                write_cache_line['data'] = "Data from Memory"
                write_cache_line['state'] = 'M'  # Transition to Modified state
                write_cache_misses += 1
            elif write_cache_line['state'] == 'M':
                output_summary.append(f"Write: Address {hex(write_address)} - Cache Hit (Modified)")
                write_cache_hits += 1

        elif write_policy == 'WT':
            output_summary.append(f"Write: Address {hex(address)} - Cache Miss (Write-Through)")
            write_cache_misses += 1

        # Implement LRU and FIFO replacement policies
        if replacement_policy == 'LRU':
            cache_order.append(set_index)
            if set_index in cache_order:
                cache_order.remove(set_index)
            cache_order.append(set_index)
        elif replacement_policy == 'FIFO':
            cache_order.append(set_index)
            if len(cache_order) > num_sets:
                removed_set = cache_order.popleft()
                cache[removed_set]['state'] = 'I'

    read_cache_hit_rate = read_cache_hits / (read_cache_hits + read_cache_misses)
    read_cache_miss_rate = read_cache_misses / (read_cache_hits + read_cache_misses)
    write_cache_hit_rate = write_cache_hits / (write_cache_hits + write_cache_misses)
    write_cache_miss_rate = write_cache_misses / (write_cache_hits + write_cache_misses)

    return output_summary, read_cache_hit_rate, read_cache_miss_rate, write_cache_hit_rate, write_cache_miss_rate

def run_simulation():
    # Get user inputs from GUI fields
    cache_size = int(cache_size_entry.get())
    block_size = int(block_size_entry.get())
    associativity = int(associativity_entry.get())
    write_policy = write_policy_var.get()
    replacement_policy = replacement_policy_var.get()
    file_path = file_path_var.get()
    
    try:
        memory_addresses = read_memory_addresses(file_path)
        output_summary, read_hit_rate, read_miss_rate, write_hit_rate, write_miss_rate = simulate_cache(
            cache_size, block_size, associativity, memory_addresses, write_policy, replacement_policy)

        result_text.config(state="normal")
        result_text.delete("1.0", "end")
        result_text.insert("end", "Simulation Summary:\n")
        result_text.insert("end", f"Read Cache Hit Rate: {read_hit_rate:.2%}\n")
        result_text.insert("end", f"Read Cache Miss Rate: {read_miss_rate:.2%}\n")
        result_text.insert("end", f"Write Cache Hit Rate: {write_hit_rate:.2%}\n")
        result_text.insert("end", f"Write Cache Miss Rate: {write_miss_rate:.2%}\n\n")
        for line in output_summary:
            result_text.insert("end", line + "\n")
        result_text.config(state="disabled")
    except Exception as e:
        result_text.config(state="normal")
        result_text.delete("1.0", "end")
        result_text.insert("end", f"An error occurred: {str(e)}")
        result_text.config(state="disabled")

# Create a GUI window
app = tk.Tk()
app.title("MSI Cache Simulator")
app.geometry("800x600")

# Create GUI elements (labels, entries, buttons)
cache_size_label = tk.Label(app, text="Cache Size (KB):")
cache_size_entry = tk.Entry(app)

block_size_label = tk.Label(app, text="Block Size (bytes):")
block_size_entry = tk.Entry(app)

associativity_label = tk.Label(app, text="Associativity:")
associativity_entry = tk.Entry(app)

file_label = tk.Label(app, text="Select Memory Addresses File:")
file_path_var = tk.StringVar()
file_entry = tk.Entry(app, textvariable=file_path_var, state="readonly")
file_browse_button = tk.Button(app, text="Browse", command=lambda: file_path_var.set(filedialog.askopenfilename()))

write_policy_label = tk.Label(app, text="Write Policy:")
write_policy_var = tk.StringVar(value="WB")
write_policy_wb = tk.Radiobutton(app, text="Write-Back (WB)", variable=write_policy_var, value="WB")
write_policy_wt = tk.Radiobutton(app, text="Write-Through (WT)", variable=write_policy_var, value="WT")

replacement_policy_label = tk.Label(app, text="Replacement Policy:")
replacement_policy_var = tk.StringVar(value="LRU")
replacement_policy_lru = tk.Radiobutton(app, text="LRU", variable=replacement_policy_var, value="LRU")
replacement_policy_fifo = tk.Radiobutton(app, text="FIFO", variable=replacement_policy_var, value="FIFO")

simulate_button = tk.Button(app, text="Simulate", command=run_simulation)

result_text = tk.Text(app, state="normal", height=26, width=88)
result_text.grid(row=8, column=0, columnspan=3)

# Place GUI elements on the window using grid
cache_size_label.grid(row=0, column=0)
cache_size_entry.grid(row=0, column=1)

block_size_label.grid(row=1, column=0)
block_size_entry.grid(row=1, column=1)

associativity_label.grid(row=2, column=0)
associativity_entry.grid(row=2, column=1)

file_label.grid(row=3, column=0)
file_entry.grid(row=3, column=1)
file_browse_button.grid(row=3, column=2)

write_policy_label.grid(row=4, column=0)
write_policy_wb.grid(row=4, column=1)
write_policy_wt.grid(row=4, column=2)

replacement_policy_label.grid(row=5, column=0)
replacement_policy_lru.grid(row=5, column=1)
replacement_policy_fifo.grid(row=5, column=2)

simulate_button.grid(row=6, column=0, columnspan=3)

# Run the GUI application
app.mainloop()


In [4]:
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox

def read_memory_addresses(filename):
    addresses = []
    with open(filename, 'r') as file:
        for line in file:
            address = int(line, 16)
            addresses.append(address)
    return addresses

def simulate_cache(cache_size, block_size, associativity, memory_addresses, write_policy, replacement_policy):
    num_sets = (cache_size * 1024) // (block_size * associativity)
    cache = [{'state': 'I', 'data': None} for _ in range(num_sets)]
    output_summary = []
    read_cache_hits = 0
    read_cache_misses = 0
    write_cache_hits = 0
    write_cache_misses = 0
    cache_order = [set_index for set_index in range(num_sets)]  # Maintain cache line order for replacement

    for address in memory_addresses:
        block_index = address // block_size
        set_index = block_index % num_sets
        cache_line = cache[set_index]

        if cache_line['state'] == 'I':
            output_summary.append(f"Read: Address {hex(address)} - Cache Miss")
            cache_line['data'] = "Data from Memory"
            cache_line['state'] = 'S'  # Transition to Shared state
            read_cache_misses += 1
        elif cache_line['state'] == 'S':
            output_summary.append(f"Read: Address {hex(address)} - Cache Hit (Shared)")
            read_cache_hits += 1
        elif cache_line['state'] == 'M':
            output_summary.append(f"Read: Address {hex(address)} - Cache Hit (Modified)")
            read_cache_hits += 1

        if write_policy == 'WB':
            write_address = address
            write_block_index = write_address // block_size
            write_set_index = write_block_index % num_sets
            write_cache_line = cache[write_set_index]

            if write_cache_line['state'] == 'I':
                output_summary.append(f"Write: Address {hex(write_address)} - Cache Miss")
                write_cache_line['data'] = "Data from Memory"
                write_cache_line['state'] = 'M'  # Transition to Modified state
                write_cache_misses += 1
            elif write_cache_line['state'] == 'S':
                output_summary.append(f"Write: Address {hex(write_address)} - Cache Miss")
                write_cache_line['data'] = "Data from Memory"
                write_cache_line['state'] = 'M'  # Transition to Modified state
                write_cache_misses += 1
            elif write_cache_line['state'] == 'M':
                output_summary.append(f"Write: Address {hex(write_address)} - Cache Hit (Modified)")
                write_cache_hits += 1

        elif write_policy == 'WT':
            output_summary.append(f"Write: Address {hex(address)} - Cache Miss (Write-Through)")
            write_cache_misses += 1

        if replacement_policy == 'LRU':
            # Update cache line access time for LRU
            cache_order.remove(set_index)
            cache_order.append(set_index)
        elif replacement_policy == 'FIFO':
            # Use the next set for replacement in FIFO
            next_set = cache_order.pop(0)
            cache_order.append(next_set)
            set_index = next_set

        # Replace the cache line
        cache_line_to_replace = cache[set_index]
        cache_line_to_replace['state'] = 'I'
        cache_line_to_replace['data'] = None

    read_cache_hit_rate = read_cache_hits / (read_cache_hits + read_cache_misses)
    read_cache_miss_rate = read_cache_misses / (read_cache_hits + read_cache_misses)
    write_cache_hit_rate = write_cache_hits / (write_cache_hits + write_cache_misses)
    write_cache_miss_rate = write_cache_misses / (write_cache_hits + write_cache_misses)

    return output_summary, read_cache_hit_rate, read_cache_miss_rate, write_cache_hit_rate, write_cache_miss_rate

def run_simulation():
    l1_cache_size = int(l1_cache_size_entry.get())
    l1_block_size = int(l1_block_size_entry.get())
    l1_associativity = int(l1_associativity_entry.get())
    l2_cache_size = int(l2_cache_size_entry.get())
    l2_block_size = int(l2_block_size_entry.get())
    l2_associativity = int(l2_associativity_entry.get())
    write_policy = write_policy_var.get()
    replacement_policy = replacement_policy_var.get()
    file_path = file_path_var.get()

    try:
        memory_addresses = read_memory_addresses(file_path)
        l1_output_summary, l1_read_hit_rate, l1_read_miss_rate, l1_write_hit_rate, l1_write_miss_rate = simulate_cache(
            l1_cache_size, l1_block_size, l1_associativity, memory_addresses, write_policy, replacement_policy)
        l2_output_summary, l2_read_hit_rate, l2_read_miss_rate, l2_write_hit_rate, l2_write_miss_rate = simulate_cache(
            l2_cache_size, l2_block_size, l2_associativity, memory_addresses, write_policy, replacement_policy)

        result_text.config(state="normal")
        result_text.delete("1.0", "end")
        result_text.insert("end", "L1 Cache Simulation Summary:\n")
        result_text.insert("end", f"L1 Read Cache Hit Rate: {l1_read_hit_rate:.2%}\n")
        result_text.insert("end", f"L1 Read Cache Miss Rate: {l1_read_miss_rate:.2%}\n")
        result_text.insert("end", f"L1 Write Cache Hit Rate: {l1_write_hit_rate:.2%}\n")
        result_text.insert("end", f"L1 Write Cache Miss Rate: {l1_write_miss_rate:.2%}\n\n")

        result_text.insert("end", "L2 Cache Simulation Summary:\n")
        result_text.insert("end", f"L2 Read Cache Hit Rate: {l2_read_hit_rate:.2%}\n")
        result_text.insert("end", f"L2 Read Cache Miss Rate: {l2_read_miss_rate:.2%}\n")
        result_text.insert("end", f"L2 Write Cache Hit Rate: {l2_write_hit_rate:.2%}\n")
        result_text.insert("end", f"L2 Write Cache Miss Rate: {l2_write_miss_rate:.2%}\n\n")

        for line in l1_output_summary:
            result_text.insert("end", line + "\n")

        for line in l2_output_summary:
            result_text.insert("end", line + "\n")

        result_text.config(state="disabled")
    except Exception as e:
        result_text.config(state="normal")
        result_text.delete("1.0", "end")
        result_text.insert("end", f"An error occurred: {str(e)}")
        result_text.config(state="disabled")

app = tk.Tk()
app.title("Two-Level Cache Simulator with MSI Protocol")
app.geometry("940x800")

# Create GUI elements for L1 cache
l1_cache_size_label = tk.Label(app, text="L1 Cache Size (KB):")
l1_cache_size_entry = tk.Entry(app)
l1_block_size_label = tk.Label(app, text="L1 Block Size (bytes):")
l1_block_size_entry = tk.Entry(app)
l1_associativity_label = tk.Label(app, text="L1 Associativity:")
l1_associativity_entry = tk.Entry(app)

# Place L1 cache GUI elements
l1_cache_size_label.grid(row=0, column=0, padx=10, pady=5)
l1_cache_size_entry.grid(row=0, column=1, padx=10, pady=5)
l1_block_size_label.grid(row=1, column=0, padx=10, pady=5)
l1_block_size_entry.grid(row=1, column=1, padx=10, pady=5)
l1_associativity_label.grid(row=2, column=0, padx=10, pady=5)
l1_associativity_entry.grid(row=2, column=1, padx=10, pady=5)

# Create GUI elements for L2 cache
l2_cache_size_label = tk.Label(app, text="L2 Cache Size (KB):")
l2_cache_size_entry = tk.Entry(app)
l2_block_size_label = tk.Label(app, text="L2 Block Size (bytes):")
l2_block_size_entry = tk.Entry(app)
l2_associativity_label = tk.Label(app, text="L2 Associativity:")
l2_associativity_entry = tk.Entry(app)

# Place L2 cache GUI elements
l2_cache_size_label.grid(row=3, column=0, padx=10, pady=5)
l2_cache_size_entry.grid(row=3, column=1, padx=10, pady=5)
l2_block_size_label.grid(row=4, column=0, padx=10, pady=5)
l2_block_size_entry.grid(row=4, column=1, padx=10, pady=5)
l2_associativity_label.grid(row=5, column=0, padx=10, pady=5)
l2_associativity_entry.grid(row=5, column=1, padx=10, pady=5)

# Create GUI elements for file selection, write policy, and replacement policy
file_label = tk.Label(app, text="Select Memory Addresses File:")
file_path_var = tk.StringVar()
file_entry = tk.Entry(app, textvariable=file_path_var, state="readonly", width=50)
file_browse_button = tk.Button(app, text="Browse", command=lambda: file_path_var.set(filedialog.askopenfilename()))

write_policy_label = tk.Label(app, text="Write Policy:")
write_policy_var = tk.StringVar(value="WB")
write_policy_wb = tk.Radiobutton(app, text="Write-Back (WB)", variable=write_policy_var, value="WB")
write_policy_wt = tk.Radiobutton(app, text="Write-Through (WT)", variable=write_policy_var, value="WT")

replacement_policy_label = tk.Label(app, text="Replacement Policy:")
replacement_policy_var = tk.StringVar(value="LRU")
replacement_policy_lru = tk.Radiobutton(app, text="LRU", variable=replacement_policy_var, value="LRU")
replacement_policy_fifo = tk.Radiobutton(app, text="FIFO", variable=replacement_policy_var, value="FIFO")

simulate_button = tk.Button(app, text="Simulate", command=run_simulation)

result_text = tk.Text(app, state="normal", height=25, width=95)
result_text.grid(row=10, column=0, columnspan=3)

# Place GUI elements on the window
file_label.grid(row=6, column=0, padx=10, pady=5)
file_entry.grid(row=6, column=1, columnspan=2, padx=10, pady=5)
file_browse_button.grid(row=6, column=3, padx=10, pady=5)
write_policy_label.grid(row=7, column=0, padx=10, pady=5)
write_policy_wb.grid(row=7, column=1, padx=10, pady=5)
write_policy_wt.grid(row=7, column=2, padx=10, pady=5)
replacement_policy_label.grid(row=8, column=0, padx=10, pady=5)
replacement_policy_lru.grid(row=8, column=1, padx=10, pady=5)
replacement_policy_fifo.grid(row=8, column=2, padx=10, pady=5)
simulate_button.grid(row=9, column=0, columnspan=3, padx=10, pady=10)

app.mainloop()